<a href="https://colab.research.google.com/github/RozitaAbdoli/credit_default_mining/blob/main/Stats_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Perform statistical significance tests by 5 iterations of 10-fold CV measuring the below 3 performance metrics and comparing model performances using the Kruskal-Wallis test to see whether model performances are statistically different from each other (0.05 significance level), followed by the post-hoc Dunn's test pairwise comparison if the Kruskal-Wallis test shows that there is a statistically significant difference:    

* F1-score (harmonic mean of precision and recall)
* ROC AUC (measures model's discriminative capability)
* Brier score (measures probability prediction of models)

In [ ]:
from scipy.stats import kruskal
!pip install scikit-posthocs
import scikit_posthocs as sp

import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

from sklearn.pipeline import Pipeline

from sklearn import preprocessing
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.ensemble import VotingClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_curve, roc_auc_score, f1_score, brier_score_loss

import time

from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

     |████████████████████████████████| 43 kB 1.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-posthocs: filename=scikit_posthocs-0.6.7-py3-none-any.whl size=37902 sha256=83e45d4981ac1ef6a10171fc2cfa2dbdd3b87bc46754c7e287d8fa97f9b93082
  Stored in directory: /root/.cache/pip/wheels/b8/21/e6/f39794d4a6ee3d3cc5146ca80b5cd949452ad4a8fde9f6b9fc
Successfully built scikit-posthocs


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#Load the dataset into pandas DataFrame
df = pd.read_csv("/content/drive/MyDrive/Capstone_project/v2_credit_default.csv")

## 1. F1-score

In [ ]:
# get the dataset
def get_dataset():
	X, y = df.drop(['Default'], axis=1), df['Default']
	return X, y

# define the base models for the heterogenous ensemble
base_models = list()
base_models.append(('RF', RandomForestClassifier()))
# base_models.append(('AB', AdaBoostClassifier()))
base_models.append(('GB', GradientBoostingClassifier()))
base_models.append(('XGB', xgb.XGBClassifier()))
 
 
# get a list of models to evaluate
def get_models():
	models = dict()
	models['Logistic Regression'] = LogisticRegression(random_state=1, C= 50, penalty= 'l1', solver= 'liblinear')
	models['KNN'] = KNeighborsClassifier()
	models['Naive Bayes'] = GaussianNB()
	# models['SVM_RBF'] = svm.SVC(kernel ='rbf', probability=True)		##omitted because it was runtime prohibitive
	models['Decision Tree'] = DecisionTreeClassifier()
	models['Random Forest'] = RandomForestClassifier()
	models['AdaBoost'] = AdaBoostClassifier()
	models['GradientBoost'] = GradientBoostingClassifier()
	models['XGBoost'] = XGBClassifier()
	models['Heterogeneous_ensemble'] = VotingClassifier(estimators=base_models, voting='soft') 
	return models
	
# evaluate models by F1-score using cross-validation
def evaluate_model_f1(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
	scores = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1, error_score='raise')
	return scores
 
# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results_f1, names = list(), list()
for name, model in models.items():
	scores = evaluate_model_f1(model, X, y)
	results_f1.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>Logistic Regression 0.359 (0.020)


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


>KNN 0.251 (0.016)
>Naive Bayes 0.387 (0.005)
>Decision Tree 0.401 (0.016)
>Random Forest 0.469 (0.016)
>AdaBoost 0.436 (0.021)
>GradientBoost 0.475 (0.018)
>XGBoost 0.473 (0.018)
>Heterogeneous_ensemble 0.476 (0.018)


In [ ]:
# First the Kruskal Wallis one-way analysis of variance by ranks (not assuming normal distribution)
stat, p = kruskal(results_f1[0], results_f1[1], results_f1[2], results_f1[3], results_f1[4], results_f1[5], results_f1[6], results_f1[7], results_f1[8])
print('Kruskal-Wallis Statistic=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05      # set significance level
if p > alpha:
	print('Same distributions (fail to reject H0)')
else:
	print('Different distributions (reject H0), proceed to post-hoc')
# post hoc Dunn's test to do pairwise comparisons
# Note: conventional p-value suffers from high Type I error, instead report the adjusted p-value (APV).
#Bonferroni correction can over-correct for type I error, and is sometimes viewed as too conservative. Holm’s sequential Bonferroni post hoc test is a less strict correction for multiple comparisons. 
f1_df = sp.posthoc_dunn(results_f1, p_adjust = 'holm')
#update column names
f1_df.rename(columns={1:'LR', 2:'KNN', 3:'NB', 4:'DT', 5:'RF', 6:'AdaBoost', 7:'GBoost', 8:'XGBoost', 9:'het_ensemble'}, inplace=True)
#update index names
f1_df.rename(index={1:'LR', 2:'KNN', 3:'NB', 4:'DT', 5:'RF', 6:'AdaBoost', 7:'GBoost', 8:'XGBoost', 9:'heterogeneous_ensemble'}, inplace=True)
f1_df

Kruskal-Wallis Statistic=387.268, p=0.000
Different distributions (reject H0), proceed to post-hoc


,LR,KNN,NB,DT,RF,AdaBoost,GBoost,XGBoost,het_ensemble
LR,1.000000e+00,2.517662e-01,4.272762e-01,1.466111e-02,7.031718e-20,3.391753e-08,1.572396e-23,1.409295e-22,1.907507e-24
KNN,2.517662e-01,1.000000e+00,5.459336e-04,1.157045e-06,5.161671e-30,5.567441e-15,1.808334e-34,2.604244e-33,1.402558e-35
NB,4.272762e-01,5.459336e-04,1.000000e+00,1.000000e+00,1.017959e-12,5.793082e-04,1.205728e-15,6.847989e-15,2.167571e-16
DT,1.466111e-02,1.157045e-06,1.000000e+00,1.000000e+00,6.941390e-09,4.993923e-02,2.327378e-11,1.039034e-10,5.448051e-12
RF,7.031718e-20,5.161671e-30,1.017959e-12,6.941390e-09,1.000000e+00,6.275331e-03,1.000000e+00,1.000000e+00,1.000000e+00
AdaBoost,3.391753e-08,5.567441e-15,5.793082e-04,4.993923e-02,6.275331e-03,1.000000e+00,2.549828e-04,5.793082e-04,1.057663e-04
GBoost,1.572396e-23,1.808334e-34,1.205728e-15,2.327378e-11,1.000000e+00,2.549828e-04,1.000000e+00,1.000000e+00,1.000000e+00
XGBoost,1.409295e-22,2.604244e-33,6.847989e-15,1.039034e-10,1.000000e+00,5.793082e-04,1.000000e+00,1.000000e+00,1.000000e+00
heterogeneous_ensemble,1.907507e-24,1.402558e-35,2.167571e-16,5.448051e-12,1.000000e+00,1.057663e-04,1.000000e+00,1.000000e+00,1.000000e+00


## 2. ROC AUC 

In [ ]:
# evaluate models by F1-score using cross-validation
def evaluate_model_auc(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
	scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1, error_score='raise')
	return scores
 
# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results_auc, names = list(), list()
for name, model in models.items():
	scores = evaluate_model_auc(model, X, y)
	results_auc.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>Logistic Regression 0.723 (0.011)
>KNN 0.609 (0.011)
>Naive Bayes 0.672 (0.014)
>Decision Tree 0.615 (0.009)
>Random Forest 0.763 (0.009)
>AdaBoost 0.774 (0.011)
>GradientBoost 0.782 (0.010)
>XGBoost 0.783 (0.010)
>Heterogeneous_ensemble 0.783 (0.010)


In [ ]:
# Kruskal Wallis one-way analysis of variance
stat, p = kruskal(results_auc[0], results_auc[1], results_auc[2], results_auc[3], results_auc[4], results_auc[5], results_auc[6], results_auc[7], results_auc[8])
print('Kruskal-Wallis Statistic=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05      # set significance level
if p > alpha:
	print('Same distributions (fail to reject H0)')
else:
	print('Different distributions (reject H0), proceed to post-hoc')
# post hoc Dunn's test for pairwise comparisons
auc_df = sp.posthoc_dunn(results_auc, p_adjust = 'holm')       #Adjusted P Values (APVs)
#update column names
auc_df.rename(columns={1:'LR', 2:'KNN', 3:'NB', 4:'DT', 5:'RF', 6:'AdaBoost', 7:'GBoost', 8:'XGBoost', 9:'het_ensemble'}, inplace=True)
#update index names
auc_df.rename(index={1:'LR', 2:'KNN', 3:'NB', 4:'DT', 5:'RF', 6:'AdaBoost', 7:'GBoost', 8:'XGBoost', 9:'heterogeneous_ensemble'}, inplace=True)
auc_df

Kruskal-Wallis Statistic=393.331, p=0.000
Different distributions (reject H0), proceed to post-hoc


,LR,KNN,NB,DT,RF,AdaBoost,GBoost,XGBoost,het_ensemble
LR,1.000000e+00,6.057962e-06,3.970576e-01,1.060681e-04,5.775082e-02,6.057962e-06,9.098132e-11,8.569407e-11,3.435432e-11
KNN,6.057962e-06,1.000000e+00,1.705155e-02,1.000000e+00,7.226400e-14,5.032532e-23,9.119529e-32,7.792187e-32,1.441786e-32
NB,3.970576e-01,1.705155e-02,1.000000e+00,9.560034e-02,4.311845e-05,4.680414e-11,2.652189e-17,2.400002e-17,6.986739e-18
DT,1.060681e-04,1.000000e+00,9.560034e-02,1.000000e+00,7.086667e-12,1.961526e-20,9.809055e-29,8.481764e-29,1.710851e-29
RF,5.775082e-02,7.226400e-14,4.311845e-05,7.086667e-12,1.000000e+00,1.827369e-01,4.797340e-04,4.797340e-04,2.755913e-04
AdaBoost,6.057962e-06,5.032532e-23,4.680414e-11,1.961526e-20,1.827369e-01,1.000000e+00,4.103631e-01,4.103631e-01,3.970576e-01
GBoost,9.098132e-11,9.119529e-32,2.652189e-17,9.809055e-29,4.797340e-04,4.103631e-01,1.000000e+00,1.000000e+00,1.000000e+00
XGBoost,8.569407e-11,7.792187e-32,2.400002e-17,8.481764e-29,4.797340e-04,4.103631e-01,1.000000e+00,1.000000e+00,1.000000e+00
heterogeneous_ensemble,3.435432e-11,1.441786e-32,6.986739e-18,1.710851e-29,2.755913e-04,3.970576e-01,1.000000e+00,1.000000e+00,1.000000e+00


## 3. Brier Score

In [ ]:
# evaluate models by Brier score loss using cross-validation
def evaluate_model_bs(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
	scores = cross_val_score(model, X, y, scoring='neg_brier_score', cv=cv, n_jobs=-1, error_score='raise')
	return scores
 
# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results_bs, names = list(), list()
for name, model in models.items():
	scores = evaluate_model_bs(model, X, y)
	results_bs.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>Logistic Regression -0.145 (0.002)
>KNN -0.186 (0.003)
>Naive Bayes -0.406 (0.020)
>Decision Tree -0.273 (0.009)
>Random Forest -0.138 (0.003)
>AdaBoost -0.244 (0.000)
>GradientBoost -0.134 (0.003)
>XGBoost -0.134 (0.003)
>Heterogeneous_ensemble -0.134 (0.003)


In [ ]:
# Kruskal Wallis one-way analysis of variance
stat, p = kruskal(results_bs[0], results_bs[1], results_bs[2], results_bs[3], results_bs[4], results_bs[5], results_bs[6], results_bs[7], results_bs[8])
print('Kruskal-Wallis Statistic=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05      # set significance level
if p > alpha:
	print('Same distributions (fail to reject H0)')
else:
	print('Different distributions (reject H0), proceed to post-hoc')
# post hoc Dunn's test for pairwise comparisons
bs_df = sp.posthoc_dunn(results_bs, p_adjust = 'holm')       #Adjusted P Values (APVs)
#update column names
bs_df.rename(columns={1:'LR', 2:'KNN', 3:'NB', 4:'DT', 5:'RF', 6:'AdaBoost', 7:'GBoost', 8:'XGBoost', 9:'het_ensemble'}, inplace=True)
#update index names
bs_df.rename(index={1:'LR', 2:'KNN', 3:'NB', 4:'DT', 5:'RF', 6:'AdaBoost', 7:'GBoost', 8:'XGBoost', 9:'heterogeneous_ensemble'}, inplace=True)
bs_df

Kruskal-Wallis Statistic=418.520, p=0.000
Different distributions (reject H0), proceed to post-hoc


,LR,KNN,NB,DT,RF,AdaBoost,GBoost,XGBoost,het_ensemble
LR,1.000000e+00,3.351069e-01,2.378329e-13,1.154409e-07,9.229843e-02,1.341892e-03,1.440966e-06,7.704865e-07,3.802111e-07
KNN,3.351069e-01,1.000000e+00,1.532310e-07,1.568893e-03,9.979953e-05,3.351069e-01,5.286619e-12,2.192093e-12,8.226893e-13
NB,2.378329e-13,1.532310e-07,1.000000e+00,3.351069e-01,2.839047e-23,1.568893e-03,1.255168e-37,2.538013e-38,4.371857e-39
DT,1.154409e-07,1.568893e-03,3.351069e-01,1.000000e+00,1.786584e-15,3.351069e-01,1.793735e-27,4.600411e-28,1.025863e-28
RF,9.229843e-02,9.979953e-05,2.839047e-23,1.786584e-15,1.000000e+00,2.735697e-09,4.325811e-02,3.253024e-02,2.303486e-02
AdaBoost,1.341892e-03,3.351069e-01,1.568893e-03,3.351069e-01,2.735697e-09,1.000000e+00,6.258264e-19,2.038375e-19,5.890722e-20
GBoost,1.440966e-06,5.286619e-12,1.255168e-37,1.793735e-27,4.325811e-02,6.258264e-19,1.000000e+00,1.000000e+00,1.000000e+00
XGBoost,7.704865e-07,2.192093e-12,2.538013e-38,4.600411e-28,3.253024e-02,2.038375e-19,1.000000e+00,1.000000e+00,1.000000e+00
heterogeneous_ensemble,3.802111e-07,8.226893e-13,4.371857e-39,1.025863e-28,2.303486e-02,5.890722e-20,1.000000e+00,1.000000e+00,1.000000e+00


#### Pairwise model comparison with the heterogenous ensemble model:

In [ ]:
# Now only looking at part of the dataframes above: the pairwise comparisons with the heterogeneous ensemble model:
f1_comp = f1_df['heterogeneous_ensemble':]
auc_comp = auc_df['heterogeneous_ensemble':]
bs_comp = bs_df['heterogeneous_ensemble':]
comp_df = pd.concat([f1_comp, auc_comp, bs_comp])
comp_df.drop(['het_ensemble'], axis=1, inplace=True)
comp_df.index = ['F1_Score', 'ROC_AUC', 'Brier_Score']
comp_df
# Can see that the GBoost and XGBoost perform as well as the ensemble model for all 3 performance metrics (0.05 significance level)
# RF performs as well as the ensemble model for F1_score
# AdaBoost performs as well as the ensemble model for ROC_AUC, but is doing a lot worse when it comes to the Brier score.

,LR,KNN,NB,DT,RF,AdaBoost,GBoost,XGBoost
F1_Score,1.907507e-24,1.402558e-35,2.167571e-16,5.448051e-12,1.000000,1.057663e-04,1.0,1.0
ROC_AUC,3.435432e-11,1.441786e-32,6.986739e-18,1.710851e-29,0.000276,3.970576e-01,1.0,1.0
Brier_Score,3.802111e-07,8.226893e-13,4.371857e-39,1.025863e-28,0.023035,5.890722e-20,1.0,1.0
